# A Simple FEA code

This solves $u''(x) + \alpha x = 0$ for $x \in [0,1]$, with boundary conditions $u(0) = u_0$ and $u(1) = u_N$.

First we're going to do some basic setup by importing useful libraries.

Then we'll define the parameter $\alpha$, which should be a positive real number, the function $f(x) = \alpha x$, and the boundary conditions.

Finally we'll defing mesh settings by setting variables <code>numnp</code> (number of nodes), <code>numel</code> (number of elements), and we'll create the array of grid points as a uniform distribution on $[0, 1]$.

In [ ]:
# Import libraries
import numpy as np                  # General calculations
import scipy.integrate as integrate # Numerical integration
import matplotlib.pyplot as plt     # Plotting

# f(x) from the ODE
alpha = 10
f = lambda x: alpha*x

# Boundary conditions
u0 = 0
uN = 1

# Global mesh data
numel = 4         # Number of elements
numnp = numel + 1  # Number of nodes
nen = 2            # Number of nodes per element

# Create the grid (nodal points)
xmin = 0
xmax = 1
x = np.linspace(xmin, xmax, numnp)

# Let's print x to show the grid points
print('x = ',x)

## Build LM array

Next we'll create the LM array which gives the global node number for each of the local element node numbers. Node numbering starts at 0 for ease of use with python indexing.

In [ ]:
# Initialize LM to the correct size
LM = np.zeros((nen, numel), dtype=int)

# Loop on elements
for e in range(numel):
    LM[0][e] = e
    LM[1][e] = e+1
    
# Let's print LM and make sure it looks right
print('LM = \n', LM)

## Assemble global stiffness and force vectors.

In the assembly phase, we initialize global stiffness $K$ and global force $F$ to the correct sizes, then we loop on elements and assemble element arrays into the global arrays.

This code uses linear basis functions. For 1D problems, we use the analytical result for the element stiffness matrix discussed in class.

The force vectors are computed using scipy's numerical integration feature, but they are pretty simple polynomials that could have been coded by hand; in real FEA codes, Gauss quadrature is typically used for any integration needed for element stiffness and force vectors

In [ ]:
# Assembly
K = np.zeros((numnp, numnp))
F = np.zeros(numnp)
for e in range(0,numel):
    
    # Get global node numbers for this element
    i1 = LM[0, e]
    i2 = LM[1, e]
    
    # Local nodal coordinates
    x1 = x[i1] # Coord for element local node 1
    x2 = x[i2] # Coord for element local node 2
    
    # Element length
    dxe = x2 - x1
    
    # Element stiffness
    ke = np.array([[1, -1],[-1, 1]])*(1/dxe)
    
    # Element shape functions
    N1 = lambda x: -(x - x2)/dxe
    N2 = lambda x:  (x - x1)/dxe
    
    # Element force vector
    fe = np.zeros(nen)
    fe[0] = integrate.quad(lambda x: N1(x)*f(x), x1, x2)[0]
    fe[1] = integrate.quad(lambda x: N2(x)*f(x), x1, x2)[0]
    
    # Assembly loop
    for i in range(nen):
        I = LM[i, e]
        F[I] += fe[i]
        for j in range(nen):
            J = LM[j, e]
            K[I,J] += ke[i,j]
    
# Check global arrays
print("Global stiffness and force:")
print("K = \n",K)
print("F = \n",F)

## Apply boundary conditions, solve system

Now we apply boundary conditions by modifying rows and columns in the stiffness matrix and force vector. 

The code below simply enforces the boundary conditions by modifying the coefficients matrix and right-hand side vector to include the equations $u(0) = u_0$ (first row) and $u(1) = u_N$ (last row). It is the simplest way to do things, and leads to an unsymmetric coefficients matrix. A better way would be to "move" known values to the right hand side, which keeps symmetry and does the solve with a smaller stiffness matrix, but this is a little more complex to set up and implement.

In [ ]:
# Modify first equation
K[0, :] = 0      # Set this row to all zeros
K[0, 0] = 1      # Set diagonal entry equal to 1
F[0]    = u0     # Set RHS value

# Modify last equation
K[-1, :] = 0     # Set this row to all zeros
K[-1, -1] = 1    # Set diagonal enetry equal to 1
F[-1] = uN       # Set RHS value

# Check the results
print("Modified stiffness and force:")
print("K = \n",K)
print("F = \n",F)

# Solve the system
u = np.linalg.solve(K, F)
print('u = \n',u)

## Plot solutions

Once the FEA solve is complete, we can plot the FEA solution. We compare to the exact solution to check out how it compares.

We can also compute the derivative of the FEA solution. Since the FEA solution is piecewise linear, the derivative is constant between grid points and is 

$\dfrac{du^e_h}{dx} = \dfrac{u_2^e - u_1^e}{x_2^e - x_1^e} = \text{constant}$ 

over each element. 

These derivative values are are calculated and added to the plot in a loop over elements.

In [ ]:
# Exact solution
xfine = np.linspace(xmin,xmax,101)
ue = xfine +(alpha/6)*xfine*(1-xfine**2)

# Exact solution derivative
due = 1 + (alpha/6)*(1-xfine**2) - 2*xfine*xfine*(alpha/6)

# Plot the FEA and exact solutions
plt.figure()
plt.plot(xfine, ue, label='exact')
plt.plot(x,u,'o--', label='FEA')
plt.title('FEA vs exact solution, Number of elements = %d' %numel)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(['Exact','FEA'])
plt.grid()

# Plot the FEA and exact solutions *derivatives*
plt.figure()
plt.plot(xfine, due)
for e in range(numel):
    # Global element nodes
    i1 = LM[0 ,e]
    i2 = LM[1, e]
    
    # Element coordinates
    x1 = x[i1]
    x2 = x[i2]
    
    # Element displacements
    u1 = u[i1]
    u2 = u[i2]
    
    # Element derivative: constant when using linear basis functions
    du_fea = (u2 - u1)/(x2 - x1)
    
    # Add to the plot
    plt.plot([x1, x2], [du_fea, du_fea], 'bo--')
    
# Plot formatting
plt.title('FEA vs exact solution *derivative*, Number of elements = %d' %numel)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(['Exact','FEA'])
plt.grid()